## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from skimage import io
import pathlib
from tqdm import tqdm
import os
import yaml
from typing import Tuple
from typing import List

import sxrd_tiff_summer_functions as analysis
import sxrd_tiff_mapper_functions as grid_analysis

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Plot a single image

A single tiff image can be plotted like this.

In [ ]:
image = io.imread("example-data/series_1/00001_00000.tiff")
plt.imshow(image, cmap='gray', vmin = 0, vmax = 500);

The intensity values at each pixel point can be written to an array and displayed like this.

In [ ]:
image_array = np.array(image, dtype='int32')
print(image_array)

## Get the max. and avg. intensity values

The maximum and average intensity values from the diffraction pattern image can easily be calculated like this.

In [ ]:
max_image = np.max(image_array)
print("The maximum intensity value is ", max_image, end = "\n\n")

avg_image = np.average(image_array)
print("The average intensity value is ", avg_image, end = "\n\n")

## Plot an intensity grid from a series of tiffs (example)

Using the `grid_tiff_intensity` function we can calculate the maximum and average intensities from a series of diffraction pattern images, and then plot the intensities as a grid of spatial (X,Y) measurement points, with a defined size.

In [ ]:
experiment_number = "103845"
input_filepath = "example-data/series_1/"
output_filepath = "example-results/"

shape_x = 5
shape_y = 2
c_map = "Reds"

grid_analysis.grid_tiff_intensity(experiment_number, input_filepath, output_filepath, 
                                  shape_x, shape_y, c_map)

## Plot an intensity grid from a series of tiffs

The file paths and user inputs for the analysis of the Diamond 2021 and 2022 experiment are included in the `yaml` configuration files, to record the inputs of the analysis.

The information in the `yaml` file can be accessed like this.

In [ ]:
# to load the Diamond 2021 analysis
config_path = "yaml/config_diamond_2022_additional_112748.yaml"
config = analysis.get_config(config_path)

We can load the parameters for our analysis using the `extract_input` and `extract_grid_input` fucntions.

In [ ]:
experiment_numbers, input_path, output_path, background_scatter_path, background_scatter_multiple = analysis.extract_input(config_path)

shape_x, shape_y, sample_numbers, start_points, end_points = grid_analysis.extract_grid_input(config_path)

Using the `grid_tiff_intensity` function we can calculate the maximum and average intensities from a series of diffraction pattern images, and then plot the intensities as a grid of spatial (X,Y) measurement points, with a defined size.

In this case, we scanned the synchrotron X-ray beam in a grid of spatial (X,Y) points to cover a matrix of 30x highly textured Ti-6Al-4V metallographic samples. By plotting the spatial distribution of the maximum and average intensity, we are hoping to find a noticeable drop in intensity, which will help us distinguish the rectangular edges of these different samples.

In [ ]:
experiment_number = str(experiment_numbers[0])
input_filepath = input_path.format(experiment_number = experiment_number)
output_filepath = output_path.format(experiment_number = experiment_number) + "intensity_maps/"
c_map = "Reds"

grid_analysis.grid_tiff_intensity(experiment_number, input_filepath, output_filepath, 
                                  shape_x, shape_y, c_map)

Recording the start and end (X,Y) points for each of the samples will then allow us to separate out the diffraction pattern images.

We can plot these points, to check we have not made an error in discerning our samples.

In [ ]:
grid_analysis.plot_grid_points(experiment_number, output_filepath,
                               start_points, end_points, 
                               shape_x, shape_y)

Now we are happy with our selection of points, we can loop through our collection of samples, as defined by the start and end points. 

The `avg_tiff_images_grid` function allows us to iterate through each of our samples in turn, to create an average tiff image from each series of patterns. This also outputs a text file recording the image numbers we have combined for each sample, to check we have averaged the correct set of images.

In [ ]:
experiment_number = str(experiment_numbers[0])
input_filepath = input_path.format(experiment_number = experiment_number)
output_filepath = output_path.format(experiment_number = experiment_number)
v_max = 500

grid_analysis.avg_tiff_images_grid(experiment_number, input_filepath, output_filepath, v_max,
                                   sample_numbers, start_points, end_points, 
                                   shape_x)

We can reload these summed/averaged tiff images to check the image properties.

*Note, check the sample number.*

In [ ]:
output_filepath = output_path.format(experiment_number = experiment_numbers[0])
image=io.imread(f"{output_filepath}sample_21/{experiment_numbers[0]}_summed1.tiff")
plt.imshow(image, cmap='gray', vmin = 0, vmax = 500);

image_array = np.array(image, dtype='int32')
print(image_array)
image_array.dtype